# Plan Semilla 3.0 - Artificial Intelligence Module

# State Space Search

In this challenge you will implement some search algorithms for navigation problems.

Your task is to create an autonomous vehicle that goes from point A to point B. You will be given some grid configurations that will simulate aerial images.

The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:


In [2]:
import numpy as np

print('Symbol', '   Meaning       ', 'Cost')
print('🛣', '       Street        ', 1)
print('🌲', '       Forest        ', 10)
print('🚸', '       School street ', 2)
print('🚦', '       Traffic light ', 3)
print('🚔', '       Police        ', 4)
print('🚘', '       Traffic       ', 6)
print('🛑', '       Stop sign     ', 5)
print('🚧', '       Construction  ', 7)
print('🏢', '       Buildings     ', 15)

COSTS = {'🛣': 1, '🌲': 10, '🚸': 2, '🚦': 3, '🚔': 4, '🚘': 6, '🛑': 5, '🚧': 7, '🏢': 15}

Symbol    Meaning        Cost
🛣        Street         1
🌲        Forest         10
🚸        School street  2
🚦        Traffic light  3
🚔        Police         4
🚘        Traffic        6
🛑        Stop sign      5
🚧        Construction   7
🏢        Buildings      15


In [3]:
test_map_1 = np.array([['🛣', '🛣', '🛣', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🛣', '🛣', '🛣'],
                       ['🌲', '🛣', '🛣', '🛣']])

map1 = np.array([['🛣', '🚸', '🛣', '🛣', '🛣', '🛣', '🛣', '🛑', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🚔', '🛣'],
                 ['🛣', '🌲', '🌲', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🌲', '🌲', '🛣', '🚘', '🛣'],
                 ['🛣', '🛑', '🛣', '🚸', '🛣', '🌲', '🛣', '🛣', '🏢', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚦'],
                 ['🌲', '🚘', '🚘', '🛣', '🛣', '🛣', '🚘', '🛣', '🏢', '🛣', '🛣', '🌲', '🛣', '🛑', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🛣', '🚸'],
                 ['🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛑', '🛣', '🛣', '🛣', '🚘', '🛣', '🛣', '🚸', '🛣', '🛣'],
                 ['🚦', '🛣', '🏢', '🚔', '🛣', '🛣', '🏢', '🚧', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚔'],
                 ['🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🏢', '🏢', '🏢', '🚸', '🛣', '🛑', '🚘', '🚔', '🛣', '🛣'],
                 ['🛣', '🛑', '🛣', '🛣', '🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🚘'],
                 ['🛣', '🛣', '🛣', '🌲', '🌲', '🚦', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚧', '🚘'],
                 ['🚘', '🚸', '🛣', '🌲', '🌲', '🛣', '🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🚦', '🚘', '🛣', '🛣'],
                 ['🚘', '🛣', '🚧', '🛣', '🛣', '🛣', '🛑', '🛣', '🚘', '🚘', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣']])

In [55]:


class Agent:

    def __init__(self, world, costs, initial_state, goal_state):

        self.moves = np.array([[0,1], [0,-1], [-1,0], [1,0]])
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.current_state = initial_state
        self.world = world
        self.costs = costs

    def possible_moves(self):

        succ_states, succ_actions, succ_costs = [],[],[]

        for move in self.moves:

            # check that the next state is in the range of the world
            #print(self.current_state)
            if 0 <= self.current_state[0] + move[0] < self.world.shape[0] and 0 <= self.current_state[1] + move[1] < self.world.shape[1]:

                # move and assign new_state to the position after moving
                new_state = [self.current_state[0] + move[0], self.current_state[1] + move[1]]

                action = [move[0], move[1]]

                cost = self.costs[self.world[new_state[0], new_state[1]]]

                succ_states.append(new_state)
                succ_actions.append(action)
                succ_costs.append(cost)

        return succ_states, succ_actions, succ_costs


    def set_current_state(self, nex_state):

        self.current_state = nex_state



class Node:

    def __init__(self, state, parent=None, action=None, cost=None):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = tuple(state)
        self.parent = parent
        self.action = action
        #self.cost = cost

        self.depth = 0
        self.path_cost = 0

        if parent:
            self.path_cost = parent.path_cost + cost
            self.depth = parent.depth + 1


    def successors(self, agent):

        succ_states, succ_actions, succ_costs = agent.possible_moves()
        children = []

        for i in range(len(succ_states)):
            children.append(Node(succ_states[i], self, succ_actions[i], succ_costs[i]))
            
        return  children

    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
        


def depth_first_graph_search(agent):

    start_node = Node(agent.current_state)
    # frontier is the stack. The function iterates while the stack has elements.
    frontier = [start_node]
    # explored are visited nodes.
    explored = set()

    while frontier:
        node = frontier[-1] #Takes out the top element from stack
        frontier.pop()
        print("Current node: ",node.state)
        if node.state == agent.goal_state:
            path_back=[node]
            while node.parent:
                path_back.append(node.parent)
                node = node.parent
            finalPath= list(reversed(path_back))
            return finalPath
        
        
        explored.add(tuple(node.state))

        # Build child nodes of the current node. Add them to the stack if not explored
        children = node.successors(agent)
        for child in children:
            if tuple(child.state) not in explored:
                frontier.append(child)
                
        
        
        if frontier:
            agent.set_current_state(frontier[-1].state)
        
    # If the stack is empty and goal node was not found, return None
    return None
        
        
        
    
    
  

def breadth_first_graph_search(agent):
    #TODO
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Queue

    explored = []
    while frontier:
        pass
    return None



def uniform_cost_search(agent):

    start_node = Node(agent.initial_state)

    frontier = [(0, start_node)]  # priority queue of nodes to explore
    explored = set()

    while frontier:
        # find the node with the lowest path cost
        min_cost = float('inf')
        min_node = None
        for cost, node in frontier:
            if cost < min_cost:
                min_cost = cost
                min_node = node
        frontier.remove((min_cost, min_node))

        if min_node.state == agent.goal_state:
            # Builds and returns the whole path from the start to the goal by following the parents
            path_back = [min_node]
            while min_node.parent:
                path_back.append(min_node.parent)
                min_node = min_node.parent
            return list(reversed(path_back))

        explored.add(min_node.state)

        #Builds child nodes of the current (min) node. Adds them to the queue.
        
        children = min_node.successors(agent)
        for child in children:            
            if child.state not in explored:                
                child_cost = child.path_cost                
                frontier.append((child_cost, child))

    return 


def order_frontier(frontier):

    costs = []
    new_list = []

    for i in range(len(frontier)):

        costs.append(frontier[i].path_cost)

    costs = np.array(costs)

    indexes = np.argsort(costs)

    for i in range(len(frontier)):

        new_list.append(frontier[indexes[i]])

    return list(reversed(new_list))





In [56]:
Ada = Agent(map1, COSTS, [0,0], [10,0])
g_node=depth_first_graph_search(Ada)
#g_node = uniform_cost_search(Ada)

if g_node is None:
    print('Did not find the goal state')
else:
    print(g_node.state)

Current node:  (0, 0)
Current node:  (1, 0)
Current node:  (2, 0)
Current node:  (3, 0)
Current node:  (4, 0)
Current node:  (5, 0)
Current node:  (6, 0)
Current node:  (7, 0)
Current node:  (8, 0)
Current node:  (9, 0)
Current node:  (10, 0)
Current node:  (11, 0)
Current node:  (12, 0)
Current node:  (12, 1)
Current node:  (11, 1)
Current node:  (10, 1)
Current node:  (9, 1)
Current node:  (8, 1)
Current node:  (7, 1)
Current node:  (6, 1)
Current node:  (5, 1)
Current node:  (4, 1)
Current node:  (3, 1)
Current node:  (2, 1)
Current node:  (1, 1)
Current node:  (0, 1)
Current node:  (0, 2)
Current node:  (1, 2)
Current node:  (2, 2)
Current node:  (3, 2)
Current node:  (4, 2)
Current node:  (5, 2)
Current node:  (6, 2)
Current node:  (7, 2)
Current node:  (8, 2)
Current node:  (9, 2)
Current node:  (10, 2)
Current node:  (11, 2)
Current node:  (12, 2)
Current node:  (12, 3)
Current node:  (11, 3)
Current node:  (10, 3)
Current node:  (9, 3)
Current node:  (8, 3)
Current node:  (7, 3

In [27]:
Ada = Agent(map1, COSTS, [0,0], [0,4])
#g_node=depth_first_graph_search(Ada)
g_node = uniform_cost_search(Ada)

if g_node is None:
    print('Did not find the goal state')
else:
    print(g_node.state)

p = g_node.path()

moves = {'01':'⏩','0-1':'⏪','-10':'⏫', '10':'⏬'}

map_copy = map1.copy()


for i in range(len(p)-1):

    ac = p[i]
    n = p[i+1]

    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]


for i in range(map_copy.shape[0]):
    print(map_copy[i,:])

UnboundLocalError: local variable 'path_back' referenced before assignment